# Analyzing Quantum Volume Routing Attempts
This notebook analyzes the effect of using multiple routing attempts and picking the bestone for the Quantum Volume algorithmat a particular depth. For a given m = depth = number of qubits, plot the HOG for a range of routing attempts.

In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")

In [ ]:
import cirq
import cirq_google

# Configuration parameters. Feel free to mess with these!
num_circuits = 10
depth = 4
max_routing_attempts = 100
routing_attempts = range(1, max_routing_attempts, 10)
repetitions = 10_000
qubits = cirq.GridQubit.rect(3,2)

def compiler(circuit):
      return cirq.optimize_for_target_gateset(circuit, gateset=cirq.CZTargetGateset())

print(f"Configuration: depth {depth} with "
      f"{num_circuits} circuits of routing attempts {routing_attempts}")

In [ ]:
# Run the Quantum Volume algorithm over the above parameters.

import numpy as np
from cirq.contrib import quantum_volume, routing

samplers = [
        cirq.DensityMatrixSimulator(noise=cirq.ConstantQubitNoiseModel(
        qubit_noise_gate=cirq.DepolarizingChannel(p=.005)))]

results = []
for r in routing_attempts:
    print(f"Running with {r} routing attempt(s)")
    results.append(quantum_volume.calculate_quantum_volume(num_circuits=num_circuits,
                            depth=depth,
                            num_qubits=depth,
                            device_graph=routing.gridqubits_to_graph_device(qubits),
                            samplers=samplers,
                            compiler=compiler,
                            random_state=np.random.RandomState(52),
                            repetitions=repetitions,
                            routing_attempts=r))

In [ ]:
# Create a chart that plots the HOG rate relative to the simulated error ratio.

from matplotlib import pyplot as plt
import statistics

probs = []
for idx, r in enumerate(routing_attempts):
    result = results[idx]
    probs.append(statistics.mean([res.sampler_result for res in result]))

fig, axs = plt.subplots()
axs.plot(routing_attempts, probs)

# Line markers for asymptotic ideal heavy output probability and the ideal Heavy
# Output Generation threshold.
axs.axhline((1 + np.log(2)) / 2,
            color='tab:green',
            label='Asymptotic ideal',
            linestyle='dashed')
axs.axhline(2 / 3, label='HOG threshold', color='k', linestyle='dotted')
axs.set_ybound(0.4, 1)
axs.set_xlabel("Number of routing attempts")
axs.set_ylabel("est. heavy output probability")
fig.suptitle(f'HOG probability by number of routing attempts for d={depth}')